In [1]:
import numpy as np
import pandas as pd

import sklearn.datasets as sd
import sklearn.preprocessing as sp
import sklearn.feature_extraction.text as ft
import sklearn.model_selection as ms
import sklearn.linear_model as lm
import sklearn.tree as st
import sklearn.ensemble as se
import sklearn.naive_bayes as nb
import sklearn.svm as svm
import sklearn.metrics as sm

import nltk.tokenize as tk
import jieba
jieba.load_userdict("mydict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\WJ\AppData\Local\Temp\jieba.cache
Loading model cost 0.745 seconds.
Prefix dict has been built successfully.


In [84]:
data = pd.read_csv("htl_all.csv")
data["label"].value_counts()
data.columns

Index(['label', 'review'], dtype='object')

In [85]:
def func(item):
    # 分词
    seg_list = jieba.cut(item)
    return " ".join(seg_list)



data["new_reviews"] = data["review"].apply(func)
data

,label,review,new_reviews
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较...","距离 川沙 公路 较近 , 但是 公交 指示 不 对 , 如果 是 "" 蔡陆线 "" 的话 ,..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!,商务 大床房 ， 房间 很大 ， 床有 2M 宽 ， 整体 感觉 经济 实惠 不错 !
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。,早餐 太 差 ， 无论 去 多少 人 ， 那边 也 不加 食品 的 。 酒店 应该 重视 一...
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...,宾馆 在 小 街道 上 ， 不大好 找 ， 但 还好 北京 热心 同胞 很多 ~ 宾馆 设施...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风","CBD 中心 , 周围 没什么 店铺 , 说 5 星 有点 勉强 . 不 知道 为什么 卫生..."
...,...,...,...
7760,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...,尼斯 酒店 的 几大 特点 ： 噪音 大 、 环境 差 、 配置 低 、 服务 效率 低 。...
7761,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...,盐城 来 了 很 多次 ， 第一次 住 盐阜 宾馆 ， 我 的确 很 失望 整个 墙壁 黑咕...
7762,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...,看 照片 觉得 还 挺不错 的 ， 又 是 4 星级 的 ， 但 入住 以后 除了 后悔 没...
7763,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...,我们 去 盐城 的 时候 那里 的 最低气温 只有 4 度 ， 晚上 冷得 要死 ， 居然 ...


In [86]:
# 整理输入集与输出集
cv = ft.CountVectorizer()
bow = cv.fit_transform(data["new_reviews"]) #按空格间隔词的形式作为输入，可转换为词袋矩阵bow

tt = ft.TfidfTransformer()
tfidf = tt.fit_transform(bow)


# 拆分测试集与训练集
train_x,test_x,train_y,test_y=\
    ms.train_test_split(tfidf,data["label"],test_size=0.1,stratify=data["label"],random_state=7)

# 交叉验证
# model_NB = lm.LogisticRegression()
# model_NB = svm.SVC(kernel="linear",C=1)
# model_NB = st.DecisionTreeClassifier(max_depth=10,min_samples_split=3,random_state=7)
# model_NB = se.RandomForestClassifier(max_depth=10,min_samples_split=3,n_estimators=200,random_state=7)

# score_f1 = ms.cross_val_score(model_NB,train_x,train_y,cv=5,scoring="f1_weighted")
# print(score_f1.mean())


#使用朴素贝叶斯模型
model_NB = nb.MultinomialNB()
score_f1 = ms.cross_val_score(model_NB,train_x,train_y,cv=5,scoring="f1_weighted")
print(score_f1.mean())



#训练模型
model_NB.fit(train_x,train_y)


#测试模型，评估模型
pred_test_y = model_NB.predict(test_x)
print(sm.classification_report(test_y,pred_test_y))



0.6700937129069996
              precision    recall  f1-score   support

           0       0.97      0.25      0.39       244
           1       0.74      1.00      0.85       533

    accuracy                           0.76       777
   macro avg       0.86      0.62      0.62       777
weighted avg       0.81      0.76      0.71       777



In [87]:
new_comment=["房间很宽敞，价格不贵",
             "睡得不舒服，早餐太贵且不好吃",
             "总体还行，考虑到携程上有红包优惠",
             "设施不齐全，下次再也来了。",
             "地板脏，空调漏水，下次不会再来了！"]

new_comment = pd.DataFrame(new_comment,columns=["reviews"])
new_comment["seg"] = new_comment["reviews"].apply(func)
new_comment

,reviews,seg
0,房间很宽敞，价格不贵,房间 很 宽敞 ， 价格 不 贵
1,睡得不舒服，早餐太贵且不好吃,睡得 不舒服 ， 早餐 太贵 且 不好吃
2,总体还行，考虑到携程上有红包优惠,总体 还 行 ， 考虑 到 携程 上 有 红包 优惠
3,设施不齐全，下次再也来了。,设施 不齐全 ， 下次 再也 来 了 。
4,地板脏，空调漏水，下次不会再来了！,地板 脏 ， 空调 漏水 ， 下次 不会 再来 了 ！


In [88]:
# 把样本按照训练时 的方式转换为tfidf矩阵，才可以交给模型训练
bow = cv.transform(new_comment["seg"])
test_data = tt.transform(bow)

pred_test_data = model_NB.predict(test_data)

pred_test_data

array([1, 1, 1, 1, 0], dtype=int64)